The problem consist of estimating the price of a European call option which underlying stock price satisfies:

$dS_t = r S_t dt + (\sigma^2+\mu^2S_t^2)^{1/2} dW_t$

In [1]:
from math import *
import numpy as np

In [2]:
S_0 = 50
r   = 0.03
K   = 50
T   = 1
m   = 30
D_t = T / m
sig = 2
mu  = 0.2
n   = 10_000

Method 1: Plain Monte Carlo

In [3]:
X = []
for k in range(n):
    S_T = S_0
    for i in range(m):
        Zk_i = np.random.normal()
        S_T += r*S_T*D_t + sqrt(sig**2 + mu**2*S_T**2)*sqrt(D_t)*Zk_i
    X += [exp(-r*T)*max(0, S_T - K)]

MC_v_chap = 0
MC_S_E    = 0
for k in range(n):
    MC_v_chap += X[k]
    MC_S_E    += (X[k])**2
MC_v_chap = MC_v_chap / n
MC_S_E    = sqrt((MC_S_E - n*(MC_v_chap**2)) / (n*(n-1)))


Method 2: Control Variate

In [4]:
X = []
Y = []
for k in range(n):
    S_T_chap = S_0
    S_T_bar  = S_0
    for i in range(m):
        Zk_i = np.random.normal()
        S_T_chap += r*S_T_chap*D_t + sqrt(sig**2 + mu**2*S_T_chap**2)*sqrt(D_t)*Zk_i
        S_T_bar  += r*S_T_bar*D_t + mu*S_T_bar*sqrt(D_t)*Zk_i
    X += [exp(-r*T)*max(0, S_T_chap - K)]
    Y += [exp(-r*T)*max(0, S_T_bar - K)]

X_bar = sum(X) / n
Y_bar = sum(Y) / n
b_star = 0
var_Y  = 0
for k in range(n):
    b_star += (X[k] - X_bar)*(Y[k] - Y_bar)
    var_Y  += (Y[k] - Y_bar)**2
b_star /= var_Y

H = []
for k in range(n):
    H += [X[k] - b_star*(Y[k] - max(0, S_0 - exp(-r*T)*K))]

CV_v_chap = 0
CV_S_E    = 0
for k in range(n):
    CV_v_chap += H[k]
    CV_S_E    += (H[k])**2
CV_v_chap = CV_v_chap / n
CV_S_E    = sqrt((CV_S_E - n*(CV_v_chap**2)) / (n*(n-1)))

In [5]:
MC_v_chap

4.848640756751602

In [6]:
CV_v_chap

1.5025865276556112

In [7]:
MC_S_E

0.07246812687078298

In [8]:
CV_S_E

6.764026318986143e-05